# Word Embeddings : le modèle Word2Vec

## Imports

In [2]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [3]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [4]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [5]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [6]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [7]:
len(bigram_phrases.vocab.keys())

258918

Prenons une clé au hasard :

In [8]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

fi


Le dictionnaire indique le score de cette coocurrence :

In [9]:
bigram_phrases.vocab[key_]

162

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [10]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [11]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [12]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [13]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [14]:
print(corpus[:100])

[['m'], ['#', 'mi', '-.......', 'nefoiifotamm', '.'], ['13', '.', '50', ';', '4', 'derniers', 'mois', ',', '15', '.', '70', ':', 'i', 'moto', 'de', 'novembre', ','], ['1', '.'], ['--', 'a', 'relut', ',', 'courant', ',', 'i3jrh', 'aaptemfere', ',', '15', '.', '78', '>>'], ["'", 'sidt', 'm', '!>>', 'a', 'ib0o', "<<'", '*', 'm0fa', '**'], ['**', '<<*-'], ['je', '%', 'i', 'i', '&', 'z', '>>', 'tsssix', '.'], ['premiere', 'installation', 'da'], ['systeme', 'kneipp'], ['dans', '1', '*', 'prou', '.'], ['rhenane', 'et', 'tallemagne', 'du', 'kord'], ['oe', '.', 'we', 'oo', '.', 'to', ';', 'poil', '.'], ['0', '.>>', '4', '9', '.', '40', ';', 'peche', '*,', 'tte', 'fi', '1', '.', '6', '*;', 'tac'], ['raisin', '*', 'noirs', '.'], ['1', '.', '80', '4', '4', '.', '00', ')', 'm', '.', 'fctanm', ',', '1', '.', '58', '4', '3jd', ';', '-', 'n', '/'], ['eb'], ['tomate', '*,', '0', '.', '50', '4', '9', '.', '80', '.'], ['m', 'sacres', '(', 'aux'], ['roux', '88', '>>,'], ['88', '<<.'], ['36', '.', '45', '/

## Entrainement d'un modèle Word2Vec sur ce corpus

In [15]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 234 ms
Wall time: 1.17 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [16]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [17]:
model = Word2Vec.load("../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [18]:
model.wv["ministre"]

array([ 0.00275539,  0.43152502,  0.0891262 , -0.2329914 ,  0.3507947 ,
       -0.7431653 ,  0.26039115,  0.0930851 ,  0.00857067,  0.31968012,
        0.53089696, -0.02572164,  0.3474256 , -0.65984607, -0.25924966,
        0.2653794 ,  0.22280288,  0.5273286 , -0.38808244, -0.41313326,
        0.1764871 ,  0.15062514, -0.30302158,  0.22767119,  0.53613317,
       -0.12255716, -0.13161968, -0.03128849,  0.57644284, -1.0341347 ,
        0.12781855,  0.20659375], dtype=float32)

### Calculer la similarité entre deux termes

In [19]:
model.wv.similarity("ministre", "roi")

0.98477685

### Chercher les mots les plus proches d'un terme donné

In [20]:
model.wv.most_similar("ministre", topn=10)

[('la_nouvelle', 0.9958145618438721),
 ('commerce', 0.9955487847328186),
 ('fois', 0.9950734972953796),
 ('campagne', 0.9945998787879944),
 ('commission', 0.9943875670433044),
 ('cinq', 0.9943042993545532),
 ('actuel', 0.994273841381073),
 ('pierre', 0.9941421151161194),
 ('route', 0.994003415107727),
 ('monde', 0.9939625263214111)]

### Faire des recherches complexes à travers l'espace vectoriel

In [21]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['belgique']))

[('soir', 0.9956175684928894), ('terre', 0.9877416491508484), ('marche', 0.987115740776062), ('france', 0.986911416053772), ('jean', 0.9868416786193848), ('duchesse', 0.9866959452629089), ('neuf', 0.9857171773910522), ('grand', 0.9854491353034973), ('demain', 0.984891951084137), ('grande', 0.9843817353248596)]


: 